In [1]:
import sys
sys.path.append('/home/potzschf/repos/')
from helperToolz.helpsters import *
from helperToolz.dicts_and_lists import *
from helperToolz.guzinski import * 
import geopandas as gpd
from collections import defaultdict
from joblib import Parallel, delayed
from datetime import datetime, timedelta
import time

In [2]:
path_b1 = '/data/Aldhani/eoagritwin/fields/output/predictions/FORCE/BRANDENBURG/masked_predictions/256_20_chips_GSA-DE_BRB-2019_cropMask_lines_touch_true_lines_touch_true_linecrop_prediction_extent.tif'
path_fields = '/data/Aldhani/eoagritwin/fields/Auxiliary/grid_search/Brandenburg/quick_n_dirty/Fields_as_mask_pixel_10.tif'

In [3]:
ds_b1 = gdal.Open(path_b1)
print(f'{ds_b1.RasterXSize}   {ds_b1.RasterYSize}')
print(ds_b1.GetGeoTransform())

ds_fields = gdal.Open(path_fields)
print(f'{ds_fields.RasterXSize}   {ds_fields.RasterYSize}')
print(ds_fields.GetGeoTransform())

29972   26904
(4376126.3630416505, 10.0, 0.0, 3404819.6079648044, 0.0, -10.0)
29972   26904
(4376126.3630416505, 10.0, 0.0, 3404819.6079648044, 0.0, -10.0)


/data/Aldhani/users/potzschf/conda/envs/workhorse/lib/python3.12/site-packages/osgeo/gdal.py:311: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


In [4]:
arr_field = ds_fields.GetRasterBand(1).ReadAsArray()
arr_b1 = ds_b1.GetRasterBand(1).ReadAsArray()

mask = (arr_field != 0) & (~np.isnan(arr_field)) & (~np.isnan(arr_b1))
flat_field = arr_field[mask].astype(int)
flat_b1 = arr_b1[mask]

In [5]:
# Create DataFrame
df = pd.DataFrame({'FieldID': flat_field, 'B1': flat_b1})

# Compute median per field
median_df = df.groupby('FieldID', as_index=False)['B1'].median()
median_df.rename(columns={'B1': 'B1_median'}, inplace=True)
median_df.to_csv('mean_b1_score_per_field.csv', index=False)

In [6]:
# Create a lookup map
median_map = dict(zip(median_df['FieldID'], median_df['B1_median']))

# Fill raster
arr_field_fill = np.vectorize(median_map.get)(arr_field)

In [ ]:
npTOdisk(arr_field_fill, path_fields,
         '/data/Aldhani/eoagritwin/fields/Auxiliary/grid_search/Brandenburg/quick_n_dirty/Fields_MEAN_B1_SCORE.tif',
         noData=0, d_type=gdal.GDT_Float32)